## הגדרות ומקדים

In [3]:


import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import folium
import fiona
from shapely.geometry import Point


In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


## פונקציות

In [5]:
def drop_geo(geoDF):
    geoDF = geoDF.drop(columns='geometry')
    return geoDF

In [6]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [7]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [8]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [9]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [10]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [11]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


In [12]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]


## להריץ תלמידים

In [13]:
%run "students_current_year_230706.ipynb"

## להעלות שכבות

In [14]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]


In [15]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(software_data_folder_location)
EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [16]:
EMP_kibolet=EMP_kibolet.fillna(0)

In [17]:
EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

In [18]:
emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

In [19]:
emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [20]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(software_data_folder_location))

In [22]:
taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms)[['Taz_num','student_nu']].pivot_table(index='Taz_num',aggfunc=sum)

In [23]:
taz=taz.fillna(0)

In [24]:
taz['pop_without_dorms_yeshiva']=taz['pop']

In [25]:
taz['pop']=taz['pop']+taz['student_dorms']

In [26]:
taz['pop_20_just_from_aprt']=taz['pop_20']

In [27]:
taz['pop_25_just_from_aprt']=taz['pop_25']

In [28]:
taz['pop_20']=taz['pop_20']+taz['student_dorms']*0.6

In [29]:
taz['pop_25']=taz['pop_25']+taz['student_dorms']*0.4

In [30]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [31]:
taz['pop_15_just_from_aprt']=taz['pop_15']

In [32]:

taz.loc[taz['main_secto']=="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

In [33]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## כימות מועסקים במרחב צתאל

In [34]:
pre_woman=0.5

In [35]:
pre_man=1-pre_woman

In [36]:
work_age=[ 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60']

In [37]:
under_work_age=[ 'pop_15', 'pop_20']

In [38]:
over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

In [39]:
taz['work_age']=taz[work_age].sum(axis=1)

taz['under_work_age']=taz[under_work_age].sum(axis=1)

taz['over_work_age']=taz[over_work_age].sum(axis=1)

In [40]:
sector='U_Orthodox'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.75+taz['work_age']*pre_man*0.55
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.07+taz['under_work_age']*pre_man*0.09
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='Jewish'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.9+taz['work_age']*pre_man*0.92
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.20+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.2+taz['over_work_age']*pre_man*0.15

sector='Arab'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='arabs_behined_seperation_wall'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09


## חישוב אבטלה

In [41]:
taz['pop_emp_employed']=0

In [42]:
taz.loc[taz['main_secto']=="U_Orthodox",'pop_emp_employed']=taz['pop_emp']*0.97

In [43]:
taz.loc[taz['main_secto']=="Jewish",'pop_emp_employed']=taz['pop_emp']*0.98

In [44]:
taz.loc[taz['jew']==0,'pop_emp_employed']=taz['pop_emp']*0.95

## חישוב יוממות

In [45]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_data_folder_location))

In [46]:
taz['commuting']=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

In [47]:
taz['pop_emp_employed_out_of_jtmt_area']=taz['pop_emp_employed']*taz['commuting']

In [48]:
emp_left_jtmt_area=taz['pop_emp_employed_out_of_jtmt_area'].sum().item()

In [49]:
round(emp_left_jtmt_area,-3)

94000.0

In [50]:
emp_from_jtmt_area=taz['pop_emp_employed'].sum().item()-emp_left_jtmt_area

In [51]:
emp_in_jtmt_area=emp_from_jtmt_area*1.07 #יוממות פנימה מחוץ למרחב

In [52]:
round(emp_in_jtmt_area,-3)

510000.0

In [53]:
emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.95 #הפחחת עובדים ניידים

In [54]:
round(emp_in_jtmt_area_without_mobile,-3)

484000.0

In [55]:
emp_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_without_palestin_not_okev

## פיזור עוקב

In [56]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

In [57]:
EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('kayim_emp>0')

In [58]:
EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

In [59]:
taz['taz_area']=taz.area

In [60]:
emp_buffer_taz=gpd.overlay(EMP_buffer,taz.reset_index())

In [61]:
emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

In [62]:
emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

In [63]:
taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()

In [64]:
taz['okev']=0
taz.loc[(~taz.index.isin(taz_num_no_need_okev))&(taz['pop']>0)&(taz['main_secto']!="Palestinia"),'okev']=1

In [65]:
taz.loc[taz['main_secto']=="arabs_behined_seperation_wall",'okev']=1

In [66]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

## מקדם עוקב

In [67]:
okev_level=0.15

In [68]:
arab_city=taz.query('jew==0 & okev==1')['aprt_20'].sum().item()

In [69]:
arab_sub=0

In [70]:
ou_sub=taz.query('main_secto=="U_Orthodox" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [71]:
ou_city=taz.query('main_secto=="U_Orthodox" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [72]:
general_city=taz.query('main_secto=="Jewish" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [73]:
general_sub=taz.query('main_secto=="Jewish" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [74]:
base_okev_factor=(emp_okev-(okev_level*general_city-okev_level*(ou_sub+arab_city)-2*okev_level*arab_sub))/(general_sub+general_city+ou_city+ou_sub+arab_city+arab_sub)

In [75]:
base_okev_factor=round(base_okev_factor,3)

In [76]:
data = [['Jewish', base_okev_factor,base_okev_factor+okev_level],['arab', base_okev_factor-okev_level,base_okev_factor], ['U_Orthodox',base_okev_factor-2*okev_level,base_okev_factor-okev_level]]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['sector', 'sub','city'])
okev_factor=df.set_index('sector')
okev_factor.to_excel(r'{}\Intermediates\okev_factor.xlsx'.format(software_data_folder_location))

In [77]:
taz['emp_okev']=0

In [78]:
taz.loc[(taz['jew']==0)&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['arab','city']

In [79]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','sub']

In [80]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','city']

In [81]:
taz.loc[(taz['main_secto']=="Jewish")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','city']

In [82]:
taz.loc[(taz['main_secto']=="Jewish")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','sub']

In [83]:
taz.emp_okev.sum().item()-emp_okev

-1816.5115199011052

## פיצול שכבת לא עוקב לאזורי תנועה

In [84]:
EMP_kibolet['emp_area']=EMP_kibolet.area

In [85]:
EMP_kibolet['ID']=EMP_kibolet.index

In [86]:
EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)

C:\Users\gidon\AppData\Local\Temp\ipykernel_21236\35171637.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 279 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)


In [87]:
EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

In [88]:
EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

In [89]:
EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

In [90]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)').drop(columns='geometry_buff')

In [91]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

In [93]:
EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID')['small_area'].sum()

In [94]:
EMP_kibolet_by_taz[['id_area_for_pre']]

,id_area_for_pre
ID,
46,101590.097
0,480407.754
0,480407.754
22,78773.513
2,1077383.721
2,1077383.721
2,1077383.721
2,1077383.721
2,1077383.721


In [95]:
EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

In [96]:
EMP_kibolet_by_taz['emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['prec_from_id']

In [98]:
taz['emp_not_okev']=drop_geo(EMP_kibolet_by_taz).pivot_table(index='Taz_num',aggfunc=sum)[['emp_without_palestin_in_taz']]

In [99]:
taz=taz.fillna(0)

In [100]:
taz['total_emp']=taz['emp_not_okev']+taz['emp_okev']+taz['emp_Education']

In [101]:
taz.query('main_secto!="Palestinian"')['total_emp'].sum()-emp_in_jtmt_area_without_mobile

-1816.5115199011052

## תוצאות

In [104]:
taz.groupby(by='zonetype')[['total_emp']].sum()#.loc[['Jerusalem','Judea and Samaria']]

,total_emp
zonetype,
Jerusalem,372287.064
Judea and Samaria,82187.101
Palestinian,18.000
Ramla,28207.725


In [107]:
taz.groupby(by='Muni_Heb')[['total_emp']].sum().loc[['ירושלים','בית שמש','ביתר עילית','מודיעין - מכבים - רעות','מעלה אדומים','מודיעין עילית']]

,total_emp
Muni_Heb,
ירושלים,324011.114
בית שמש,26993.914
ביתר עילית,7183.027
מודיעין - מכבים - רעות,25599.930
מעלה אדומים,7633.667
מודיעין עילית,10778.885


In [109]:
taz.groupby(by=['Muni_Heb','main_secto'])[['total_emp']].sum().loc[['ירושלים']]

total_emp
Muni_Heb main_secto                              
ירושלים  Arab                           57190.378
         Jewish                        197846.853
         U_Orthodox                     55886.933
         arabs_behined_seperation_wall  13086.951

## התפלגות ענפי תעסוקה

In [110]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_data_folder_location)).fillna(0)

In [111]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [112]:
emp_category_type[col]=emp_category_type[col]/100

In [113]:
emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

In [114]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [115]:
taz.join(emp_category_type[col]).query('(emp_not_okev>0 | emp_okev>0 )& agri.isna() & main_secto!="Palestinian" ')

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,commuting,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [116]:
taz=taz.join(emp_category_type[col]).fillna(0)

In [117]:
for i in col:
    taz['{}'.format(i)]=taz['{}'.format(i)]*(taz['emp_not_okev']+taz['emp_okev'])

In [118]:
taz['check_emp_sum']=taz[col].sum(axis=1)-(taz['emp_not_okev']+taz['emp_okev'])

In [119]:
taz.query('check_emp_sum>10 |check_emp_sum<-10')

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,commuting,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public,check_emp_sum
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת סטודנטים אל כמות משקי בית

In [120]:
taz=taz.fillna(0)

In [121]:
taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [122]:
taz['hh']=taz['hh']+taz['yeshiva_dorms_pop_sum']

In [123]:
taz.loc[taz['main_secto']!="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

## הוספת אוכלוסיה פלסטינאית

In [124]:
path=r'{}\background_files'.format(software_data_folder_location)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [125]:
col=['pop',
'pop_0',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_5',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'emp_Education',
'pop_emp_employed',
'total_emp',
'agri',
'Indus',
'Com_hotel',
'Business',
'Public',
'hh']

In [126]:
taz.loc[taz['main_secto']=='Palestinian',col]=0

In [127]:
taz.loc[taz['main_secto']=='Palestinian','pop']=taz['pop']+taz_demo_pls_2020['pop_2020']

## ייצוא לבקרה

In [128]:
taz=taz.reset_index()

In [129]:
pre_yeshiva_from_20_30_age=taz[['student_yeshiva_with_add_from_old']].sum().sum()/taz.query('jew==1')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [130]:
pre_yeshiva_from_20_30_age

0.5261022022178202

In [131]:
taz.main_secto.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [132]:
pre_uni_from_20_30_age=taz[['uni_students']].sum().sum()/taz.query('main_secto!="Palestinian"')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [133]:
pre_uni_from_20_30_age

0.3034760359349936

In [134]:
list(taz)

['Taz_num',
 'CR_PNIM',
 'Join_Count',
 'TARGET_FID',
 'Agg_taz_nu',
 'Taz_name',
 'Arab',
 'arabs_behi',
 'Jewish',
 'Palestinia',
 'U_Orthodox',
 'main_secto',
 'ENG_NAME_n',
 'Muni_Heb',
 'Sug_Muni',
 'jeru_metro',
 'zonetype',
 'in_jerusal',
 'SCHN_NAME',
 'geometry',
 'STAT',
 'aprt_20',
 'pop',
 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop_hardi',
 'precent_of_stat_data',
 'pre_hardi',
 'hh_size',
 'pop_check',
 'student_toddlers',
 'student_gov',
 'delta_student',
 'student_demand',
 'student_demand_left',
 'student_demand_left_sum_by_muni',
 'student_demand_left_pre',
 'cbs_muni_student_left_by_pre_of_demand_left',
 'jew',
 'student_demand_pre',
 'cbs_muni_student_left_by_pre_of_demand',
 'student_for_Control',
 'student_chardi_not_gov',
 'student_arab_not_gov',
 'student',
 'uni_students',
 'emp_from_uni_student',
 'old_student_yeshiva',
 'yes

In [135]:
col=['Taz_num','Agg_taz_nu','Taz_name','Muni_Heb','jeru_metro',
 'zonetype',
 'in_jerusal','main_secto','aprt_20','student_dorms','student_yeshiva_with_add_from_old','hh','pop_without_dorms_yeshiva',
 'pop', 'hh_size','pop_emp_employed',
 'commuting','student_toddlers',
 'student_gov','cbs_muni_student_left_by_pre_of_demand_left','cbs_muni_student_left_by_pre_of_demand','student_chardi_not_gov',
 'student_arab_not_gov','student','uni_students',
 'student_yeshiva','yeshiva_dorms_pop_15',
 'yeshiva_dorms_pop_20',
 'yeshiva_dorms_pop_25',
 'yeshiva_dorms_pop_sum',
 'add_from_old_student_yeshiva',
 'add_from_kollim_demand', 'emp_from_uni_student','emp_from_Yeshiva_student','emp_from_student',
 'emp_Education','okev',
 'emp_okev',
 'emp_not_okev','total_emp',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

In [136]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [137]:
taz[col].to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## ייצוא בפורמט של מודלים

In [138]:
taz['yosh']=0

In [139]:
taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

In [140]:
taz['jerusalem_city']=0

In [141]:
taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

In [142]:
taz=taz.reset_index()

In [143]:
col=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'pop_emp_employed']

In [144]:
col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [145]:
taz=taz[col]

In [146]:
taz.columns=col_new_name

In [147]:
taz.to_excel(r'{}\forecast_2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## בקרות לוגיות

In [148]:
taz.loc[taz['TAZ']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [149]:
taz.yosh.unique()

array([0, 1], dtype=int64)

In [150]:
taz.in_jerusalem_metropolin.unique()

array([1., 0.])

In [151]:
taz.jerusalem_city.unique()

array([1, 0], dtype=int64)

In [152]:
taz.sector.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [153]:
col=['hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [154]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]<0]['TAZ'])

In [155]:
taz_to_check

[]

In [156]:
taz.loc[~(taz['hh_total']<=taz['pop'])]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [157]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['hh_total']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [158]:
col=['age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

In [159]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0].query('TAZ==511')#.to_excel('delet.xlsx')#[col].sum(axis=1)

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [160]:
len(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0])

0

In [161]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop']>0].loc[taz['hh_total']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [162]:
col=[
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up']

In [163]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['pop']<=0]['TAZ'])
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['hh_total']<=0]['TAZ'])

In [164]:
taz_to_check

[]

In [165]:
col=['indus',
'com_hotel',
'business',
'public',
'education',
'agri']

In [166]:
taz['emp_check']=round(taz[col].sum(axis=1)-taz['emp_tot'])

In [167]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['emp_check']!=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check


In [168]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['emp_tot']<=0]['TAZ'])

In [169]:
taz_to_check

[]

In [170]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_emp_employed']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check
